### Pandas处理数据的基本功能
主要介绍操作Series和DataFrame中的数据的基本手段
#### Series重新索引（reindex）
reindex会创建一个适应新索引的**新对象**

In [1]:
import pandas as pn

In [2]:
obj = pn.Series([4.5,5.6,3.4,2.6],index=['a','b','c','d'])

In [3]:
obj

a    4.5
b    5.6
c    3.4
d    2.6
dtype: float64

调用该Series的reindex将会根据新索引进行重排，如果某个索引值当前不存在就引入缺失值

In [4]:
obj2 = obj.reindex(['a','b','c','d','e'])

In [5]:
obj2

a    4.5
b    5.6
c    3.4
d    2.6
e    NaN
dtype: float64

#### Series插值处理

In [6]:
obj.reindex(['a','b','c','d','e'], fill_value = 0)

a    4.5
b    5.6
c    3.4
d    2.6
e    0.0
dtype: float64

对于时间序列这样的有序数据，重新索引时，可能需要做一些**插值处理**，**method选项**则可以达到此目的，例如，使用ffill可以实现**前向值填充**：

In [8]:
obj3 = pn.Series(['blue', 'purple', 'yellow'], index = [0,2,4])

In [9]:
obj3.reindex(range(6), method = 'ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

下表列出了可用的method的选项，其实我们有时需要比**前向**和**后向**填充更为精准的插值方式
![](images/20190527115725.png)
#### DataFrame重新索引
对于DataFrame，reindex可以修改列（行）索引，或两个都修改，如果仅传入一个序列，则会重新索引行

In [10]:
import numpy as np
frame = pn.DataFrame(np.arange(9).reshape((3,3)), index = ['a', 'b','d'], columns=[2001,2000,2002])

In [11]:
frame

,2001,2000,2002
a,0,1,2
b,3,4,5
d,6,7,8


In [12]:
frame2 = frame.reindex(['a', 'b','c','d'])

In [13]:
frame2

,2001,2000,2002
a,0.0,1.0,2.0
b,3.0,4.0,5.0
c,NaN,NaN,NaN
d,6.0,7.0,8.0


使用columns关键字可重新索引列：

In [19]:
state = [2001,2003,2002]

In [20]:
frame.reindex(columns=state)

,2001,2003,2002
a,0,NaN,2
b,3,NaN,5
d,6,NaN,8


#### DataFrame插值处理
也可以同时对行和列进行重新索引，而插值则只能按行应用

In [21]:
frame.reindex(index=['a', 'b','c','d'], columns=state).ffill()

,2001,2003,2002
a,0.0,NaN,2.0
b,3.0,NaN,5.0
c,3.0,NaN,5.0
d,6.0,NaN,8.0


利用ix的标签索引功能，重新索引任务可以变得更简洁(该功能将会被废除)

In [24]:
frame.loc[['a','b','c','d'], state]

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,2001,2003,2002
a,0.0,NaN,2.0
b,3.0,NaN,5.0
c,NaN,NaN,NaN
d,6.0,NaN,8.0


下面列出reindex函数的各参数及说明
![](images/7178691-efa3dbd4b83c61ec.jpg)

#### 丢弃指定轴上的项（drop方法）
丢弃某条轴上一个或多个项，只要一个索引数组或列表即可。由于需要执行一些数据整理和集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的**新对象**

In [26]:
obj = pn.Series(np.arange(5), index = ['a','b','c','d','e'])

In [27]:
new_obj = obj.drop('c')

In [28]:
new_obj

a    0
b    1
d    3
e    4
dtype: int32

对于DataFrame,可以删除任何轴上的索引值：

In [29]:
data = pn.DataFrame(np.arange(16).reshape(4,4), columns = ['a','b','c','d'], index = ['one', 'two','three','four'])

In [30]:
data.drop(['one','three'])

,a,b,c,d
two,4,5,6,7
four,12,13,14,15


In [33]:
data.drop(['a','d'],axis=1)

,b,c
one,1,2
two,5,6
three,9,10
four,13,14


In [34]:
data.drop('b', axis=1)

,a,c,d
one,0,2,3
two,4,6,7
three,8,10,11
four,12,14,15


#### 索引、选取和过滤
##### Series索引和切片
Series索引（obj[...]）类似于Numpy数组，不同在于，Series的索引值可以不为整数

In [36]:
obj_index = pn.Series(np.arange(5), index=['a','b','c','d','e'])

In [37]:
obj_index

a    0
b    1
c    2
d    3
e    4
dtype: int32

In [38]:
obj_index['b']

1

In [39]:
obj_index[1]

1

In [40]:
obj_index[1:4]

b    1
c    2
d    3
dtype: int32

In [41]:
obj_index[['a','b']]

a    0
b    1
dtype: int32

In [42]:
obj_index[[1,2]]

b    1
c    2
dtype: int32

In [43]:
obj_index[obj_index>0]

b    1
c    2
d    3
e    4
dtype: int32

Series切片不同点在于利用**标签**的切片运算，包含其末端

In [47]:
obj_index['a':'c']

a    0
b    1
c    2
dtype: int32

设置的方法：

In [48]:
obj_index['a':'c'] = 5

In [49]:
obj_index

a    5
b    5
c    5
d    3
e    4
dtype: int32

##### DataFrame索引和切片
对于DataFrame索引，其实就是获取一个或多个列

In [52]:
dataFrame = pn.DataFrame(np.arange(16).reshape([4,4]),index=['one','two','three','four'],columns=['a','b','c','d'])

In [53]:
dataFrame

,a,b,c,d
one,0,1,2,3
two,4,5,6,7
three,8,9,10,11
four,12,13,14,15


In [54]:
dataFrame['a']

one       0
two       4
three     8
four     12
Name: a, dtype: int32

In [55]:
dataFrame[['a', 'c']]

,a,c
one,0,2
two,4,6
three,8,10
four,12,14


通过切片或布尔型数组选取行：

In [56]:
dataFrame[:2]

,a,b,c,d
one,0,1,2,3
two,4,5,6,7


In [57]:
dataFrame[dataFrame['a']>5]

,a,b,c,d
three,8,9,10,11
four,12,13,14,15


布尔型DataFrame索引

通过布尔型DataFrame进行索引

In [60]:
dataFrame<5

,a,b,c,d
one,True,True,True,True
two,True,False,False,False
three,False,False,False,False
four,False,False,False,False


In [62]:
dataFrame[dataFrame<5] = 2

In [63]:
dataFrame

,a,b,c,d
one,2,2,2,2
two,2,5,6,7
three,8,9,10,11
four,12,13,14,15


这段使得DataFrame在语法上更像ndarray

##### 使用loc和iloc选择数据
针对DataFrame在行上的标签索引，将介绍特殊的索引符号**loc**和**iloc**，loc(轴标签)和iloc（整数标签）以Numpy风格的语法，从DataFrame中选出数组的行和列的子集。
1. loc通过标签选出单行、多列

In [64]:
data

,a,b,c,d
one,0,1,2,3
two,4,5,6,7
three,8,9,10,11
four,12,13,14,15


In [66]:
data.loc['two',['a','c','d']]

a    4
c    6
d    7
Name: two, dtype: int32

2. 使用iloc进行类似的数据选择

In [69]:
data.iloc[1,[0,2,3]]

a    4
c    6
d    7
Name: two, dtype: int32

In [70]:
data.iloc[2]

a     8
b     9
c    10
d    11
Name: three, dtype: int32

In [71]:
data.iloc[[1,2],[1,2,3]]

,b,c,d
two,5,6,7
three,9,10,11


3. 切片：除了单个的标签或标签列表之外，索引功能还可以用于切片

In [73]:
data.loc[:'three','c']

one       2
two       6
three    10
Name: c, dtype: int32

##### 整数索引
当你有一个包含整数的轴索引时，经常会产生一个歧义，如下:

In [76]:
seriesFix = pn.Series(np.arange(3),index=[2,3,4])

In [77]:
seriesFix

2    0
3    1
4    2
dtype: int32

In [78]:
seriesFix[2]

0

上述中，我们使用seriesFix[2]，这使得很难推断出用户所需要的索引方式（标签索引或位置索引），而对于非整数索引则不会产生歧义。

为了保持一致性，如果你有一个包含整数的轴索引，数据选择时，请始终使用标签索引

为了精确地处理，可以使用loc和iloc

In [80]:
seriesFix[:2]

2    0
3    1
dtype: int32

In [81]:
seriesFix.loc[:2]

2    0
dtype: int32

In [82]:
seriesFix.iloc[:2]

2    0
3    1
dtype: int32

#### 算术运算和数据对齐
Pandas最重要的一个功能是，它可以对不同索引的对象进行算术运算，在将对象相加时，如果存在不同的索引时，则结果是索引的并集
##### Series自动对齐操作

In [83]:
s1 = pn.Series(np.arange(5), index=['a','b','c','d','e'])

In [84]:
s1

a    0
b    1
c    2
d    3
e    4
dtype: int32

In [88]:
s2 = pn.Series([-2.1, 3, -2.6, 6], index=['a','b','c','g'])

In [89]:
s2

a   -2.1
b    3.0
c   -2.6
g    6.0
dtype: float64

In [90]:
s1+s2

a   -2.1
b    4.0
c   -0.6
d    NaN
e    NaN
g    NaN
dtype: float64

自动对齐操作在不重叠的索引处引入了NA值，缺失值会在算术运算过程中**传播**
##### DataFrame对齐操作
DataFrame对齐操作会同时发生在行和列上

In [93]:
df1 = pn.DataFrame(np.arange(9).reshape([3,3]), columns=list('abc'), index=['one','two','three'])

In [94]:
df1

,a,b,c
one,0,1,2
two,3,4,5
three,6,7,8


In [104]:
df2 = pn.DataFrame(np.arange(16).reshape([4,4]), columns=list('abcd'), index=['one','two','three','four'])

In [105]:
df2

,a,b,c,d
one,0,1,2,3
two,4,5,6,7
three,8,9,10,11
four,12,13,14,15


In [106]:
df1 + df2

,a,b,c,d
four,NaN,NaN,NaN,NaN
one,0.0,2.0,4.0,NaN
three,14.0,16.0,18.0,NaN
two,7.0,9.0,11.0,NaN


#### 在算法方法中填充值（fill_value）
在对不同索引的对象进行运算时，有时候我们希望当一个对象中某个轴标签在另一个对象中找不到时填充一个值,我们使用**add方法**传入一个**fill_value**参数

In [107]:
df1 + df2

,a,b,c,d
four,NaN,NaN,NaN,NaN
one,0.0,2.0,4.0,NaN
three,14.0,16.0,18.0,NaN
two,7.0,9.0,11.0,NaN


In [108]:
df1.add(df2, fill_value=0)

,a,b,c,d
four,12.0,13.0,14.0,15.0
one,0.0,2.0,4.0,3.0
three,14.0,16.0,18.0,11.0
two,7.0,9.0,11.0,7.0


这里面的fill_value=0不是指填充0，而是填充的值为0加上以前的值

类似的对Series或DataFrame重新索引时，也可以指定填充值

In [109]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d
one,0,1,2,0
two,3,4,5,0
three,6,7,8,0


灵活算术方法
![](images/20190605130511.jpg)

#### DataFrame和Series之间的运算
跟Numpy一样，DataFrame和Series之间的运算也有明确的规定

In [110]:
arr = np.arange(12).reshape([3,4])

In [111]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [112]:
arr[0]

array([0, 1, 2, 3])

In [113]:
arr - arr[0]

array([[0, 0, 0, 0],
       [4, 4, 4, 4],
       [8, 8, 8, 8]])

1. 在行上进行广播

这就是Numpy中的广播，DataFrame和Series之间的运算和广播类似：

In [115]:
df1

,a,b,c
one,0,1,2
two,3,4,5
three,6,7,8


In [118]:
series = df1.iloc[0]

In [119]:
series

a    0
b    1
c    2
Name: one, dtype: int32

In [117]:
df1 - series

,a,b,c
one,0,0,0
two,3,3,3
three,6,6,6


可以看到DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着一直向下广播

如果某个索引值在DataFrame或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集

In [120]:
ser2 = pn.Series(range(3), index=['b', 'c', 'd'])

In [122]:
ser2

b    0
c    1
d    2
dtype: int64

In [121]:
df1 + ser2

,a,b,c,d
one,NaN,1.0,3.0,NaN
two,NaN,4.0,6.0,NaN
three,NaN,7.0,9.0,NaN


2. 在列上进行广播

在列上进行广播，必须使用算术运算方法

In [123]:
ser3 = df1['a']

In [124]:
ser3

one      0
two      3
three    6
Name: a, dtype: int32

In [125]:
df1

,a,b,c
one,0,1,2
two,3,4,5
three,6,7,8


In [139]:
df1.sub(ser3, axis=0)

,a,b,c
one,0,1,2
two,0,1,2
three,0,1,2


传入的轴号是希望匹配的轴

上面，我们的目的是匹配DataFrame的行索引并进行广播

#### 函数应用和映射
Numpy的ufuncs（元素级数组方法）也可以操作pandas对象

In [129]:
df3 = pn.DataFrame(np.random.randn(4,3), columns=list('bde'), index=['one', 'two', 'three', 'four'])

In [132]:
df3

,b,d,e
one,0.921911,-0.612548,-1.339804
two,0.531443,0.366117,-1.811528
three,-0.746660,-1.898992,1.562868
four,-1.092723,0.010769,2.049831


In [130]:
np.abs(df3)

,b,d,e
one,0.921911,0.612548,1.339804
two,0.531443,0.366117,1.811528
three,0.746660,1.898992,1.562868
four,1.092723,0.010769,2.049831


#### apply方法
apply()方法是针对某些行或列进行操作的

In [134]:
f = lambda x : x.max() - x.min()

In [141]:
df3.apply(f)

b    2.014634
d    2.265109
e    3.861359
dtype: float64

In [136]:
df3.apply(f, axis=1)

one      2.261715
two      2.342971
three    3.461860
four     3.142554
dtype: float64

许多常见的数组统计功能都被实现成DataFrame方法（如sum和mean），因此无需使用apply方法

除标量外，传递给apply的函数可以返回由多个值组成Series:

In [142]:
def f(x):
    return pn.Series([x.min(), x.max()], index=['min', 'max'])

In [143]:
df3.apply(f)

,b,d,e
min,-1.092723,-1.898992,-1.811528
max,0.921911,0.366117,2.049831


#### applymap方法
applymap()方法则是针对所有元素进行操作的。

In [144]:
famat = lambda x : '你好%s' %x

In [145]:
df3.applymap(famat)

,b,d,e
one,你好0.9219109531606738,你好-0.6125480382970925,你好-1.3398036145722778
two,你好0.5314431471416481,你好0.3661170179228419,你好-1.8115282040746392
three,你好-0.7466597134024525,你好-1.898992065486419,你好1.5628681007282452
four,你好-1.0927233998888612,你好0.010769155577148715,你好2.0498307429535783


之所以叫applymap，是因为Series有一个用于应用元素级函数的map方法

In [146]:
df3['e'].map(famat)

one      你好-1.3398036145722778
two      你好-1.8115282040746392
three     你好1.5628681007282452
four      你好2.0498307429535783
Name: e, dtype: object

#### 排序和排名
1. 按索引排序（sort_index）：
使用**sort_index方法**对行或列索引进行排序（按字典顺序），并返回一个已排序的新对象

In [147]:
obj = pn.Series(range(4), index=['b','c','a','d'])

In [148]:
obj.sort_index()

a    2
b    0
c    1
d    3
dtype: int64

而DataFrame可以根据任意一个轴上的索引进行排序

In [149]:
frame = pn.DataFrame(np.arange(8).reshape((2,4)), index=['three', 'one'], columns=['d', 'c', 'b', 'a'])

In [150]:
frame.sort_index()

,d,c,b,a
one,4,5,6,7
three,0,1,2,3


In [151]:
frame.sort_index(axis=1)

,a,b,c,d
three,3,2,1,0
one,7,6,5,4


默认为升序排序，也可以降序

In [152]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,1,2,3
one,4,5,6,7


2. 按值排序（sort_values）：若要按值进行排序，可以使用sort_values方法：

In [153]:
obj = pn.Series([-2, 4, 1.5, 0.1])

In [158]:
obj.sort_values()

0   -2.0
3    0.1
2    1.5
1    4.0
dtype: float64

在排序时，任何缺失值默认都放到Series的末尾

In [159]:
objNa = pn.Series([4,np.NaN, 7, np.NAN, -2])

In [160]:
objNa.sort_values()

4   -2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

在DataFrame上，要想根据一个或多个列中的值进行排序，只需将一个或多个列的名字传递给**by**选项，就可以达到目的

In [176]:
frame

,d,c,b,a
three,0,16,2,3
one,4,13,6,7


In [177]:
frame.sort_values(by='c', ascending=False)

,d,c,b,a
three,0,16,2,3
one,4,13,6,7


In [178]:
frame.sort_values(by=['c', 'a'], ascending=False)

,d,c,b,a
three,0,16,2,3
one,4,13,6,7


从上面可以看出，当多列排序时，结果以第一列为准

#### 排名（rank）
排名（ranking）跟排序关系密切，且它会增设一个排名值（从1开始，一直到数组中有效数据的数量），它跟numpy.argsort产生的间接排序索引差不多，只不过它可以根据某种规则**破坏平级关系**，默认情况下，rank是通过“为各组分配一个平均排名的方式破坏平级关系”

In [193]:
objRank = pn.Series([1,3,2,4,4,4,4,5,6])

In [194]:
objRank.rank()

0    1.0
1    3.0
2    2.0
3    5.5
4    5.5
5    5.5
6    5.5
7    8.0
8    9.0
dtype: float64

也可以根据在原数据中出现的顺序给出排名

In [195]:
objRank.rank(method='first')

0    1.0
1    3.0
2    2.0
3    4.0
4    5.0
5    6.0
6    7.0
7    8.0
8    9.0
dtype: float64

降序进行排名

In [196]:
objRank.rank(ascending=False, method='max')

0    9.0
1    7.0
2    8.0
3    6.0
4    6.0
5    6.0
6    6.0
7    2.0
8    1.0
dtype: float64

下表中列出所有用于破坏平级关系的method选项
![](images/20190605130634.jpg)

DataFrame可以在列或行上计算排名

In [197]:
frame

,d,c,b,a
three,0,16,2,3
one,4,13,6,7


In [198]:
frame.rank(axis=1)

,d,c,b,a
three,1.0,4.0,2.0,3.0
one,1.0,4.0,2.0,3.0


#### 带有重复值的轴索引
虽然许多Pandas函数（如reindex）都要求标签唯一，但并不是强制性的

In [199]:
objNo = pn.Series(range(5), index=['a','a','b','b','c'])

In [200]:
objNo

a    0
a    1
b    2
b    3
c    4
dtype: int64

可以使用**is_unique属性**来判断该索引值的唯一性

In [202]:
objNo.index.is_unique

False

对于有重复值的索引，在数据选取时将有些不同，如果某个索引对应于多个值，则会返回一个Series，对应单个值，返回一个标量值

In [203]:
objNo['a']

a    0
a    1
dtype: int64

In [204]:
objNo['c']

4

对于DataFrame的行进行索引时也是如此

In [205]:
dfNo = pn.DataFrame(np.random.randn(4,3), index=['a','a','b','b'])

In [206]:
dfNo

,0,1,2
a,0.540288,-0.593436,-0.416402
a,-1.351824,0.854870,1.554062
b,2.088016,-0.328943,-0.120746
b,1.381514,-0.012996,-0.475373


In [208]:
dfNo.loc['a']

,0,1,2
a,0.540288,-0.593436,-0.416402
a,-1.351824,0.854870,1.554062
